In [ ]:
!pip install selenium beautifulsoup4


In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!pip install selenium beautifulsoup4 webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from collections import defaultdict
import datetime
import statistics
from collections import Counter
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [ ]:
# 1️ Configure Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # run in background (no GUI)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox") # Add this line for Colab environment
chrome_options.add_argument("--disable-dev-shm-usage") # Add this line for Colab environment
chrome_options.binary_location = '/usr/bin/google-chrome' # Explicitly set the path to the Chrome binary

In [ ]:


# Set up Selenium WebDriver
service = Service(ChromeDriverManager().install()) # Use webdriver_manager to install chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)


try:
    # Navigate to the AccuWeather forecast page for Attica, GA
    url = 'https://www.accuweather.com/en/us/attica/30607/weather-forecast/2139413'
    driver.get(url)

    # Wait for the page to load
    time.sleep(5)

    # Get the page source after JavaScript has loaded
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Find all daily forecast elements
    forecast_items = soup.find_all('div', class_='forecast-card')
    print(forecast_items)

    # Lists to store temperature, condition, and humidity data
    temperatures = []
    conditions = []
    humidities = []

    # Extract data from each forecast item
    for item in forecast_items:
        # Extract temperature
        temp_tag = item.find('span', class_='high')
        if temp_tag:
            temp = int(temp_tag.text.replace('°', '').strip())
            temperatures.append(temp)

        # Extract condition
        cond_tag = item.find('span', class_='phrase')
        if cond_tag:
            condition = cond_tag.text.strip()
            conditions.append(condition)

        # Extract humidity
        humidity_tag = item.find('span', class_='humidity')
        if humidity_tag:
            humidity = int(humidity_tag.text.replace('%', '').strip())
            humidities.append(humidity)

    # Calculate average temperature
    if temperatures:
        avg_temp = sum(temperatures) / len(temperatures)
    else:
        avg_temp = None

    # Identify the most common weather condition
    if conditions:
        most_common_condition = Counter(conditions).most_common(1)[0][0]
    else:
        most_common_condition = None

    # Print the analysis results
    print(f"Average Temperature: {avg_temp:.1f}°F" if avg_temp is not None else "Temperature data not available")
    print(f"Most Common Weather Condition: {most_common_condition}" if most_common_condition else "Weather condition data not available")

finally:
    driver.quit()
